In [12]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = (
    SparkSession.builder
    .master("local")
    .appName("test-pyspark-dataframe-extras")
    .getOrCreate()
)

Importing dataframe_ext will add several functions to the Pyspark DataFrame class

In [13]:
from pyspark_dataframe_extras.common import dataframe_ext

# frequencyCount() / showFrequencyCount()

In [16]:
data = [
        ("A", 1),
        ("B", 2),
        ("C", 3),
        ("C", 4),
        ("C", 5),
        ("C", 6),
        ("A", 7),
        ("A", 8),
        ("C", 9),
        ("B", 10)
    ]

df = spark.createDataFrame(data, ["letter", "extra_column"])
df.show()

+------+------------+
|letter|extra_column|
+------+------------+
|     A|           1|
|     B|           2|
|     C|           3|
|     C|           4|
|     C|           5|
|     C|           6|
|     A|           7|
|     A|           8|
|     C|           9|
|     B|          10|
+------+------------+



The **frequencyCount()** function will:
- Group rows by the `group_by` columns
- Generate a Count and Frequency value for each group
- Order the groups in descending order by frequency

In [9]:
df.frequencyCount("letter").show()

+------+-----+---------+
|letter|count|frequency|
+------+-----+---------+
|     C|    5|      0.5|
|     A|    3|      0.3|
|     B|    2|      0.2|
+------+-----+---------+



In [8]:
df.showFrequencyCount("letter")

+------+-----+---------+
|letter|count|frequency|
+------+-----+---------+
|C     |5    |0.50     |
|A     |3    |0.30     |
|B     |2    |0.20     |
+------+-----+---------+



# firstInGroup()

In [11]:
data = [
    ("George", "Washington", "1789-04-30", None),
    ("John", "Adams", "1797-03-04", "Federalist"),
    ("James", "Madison", "1809-03-04", "Democratic-Republican"),
    ("James", "Madison", "1813-03-04", "Democratic-Republican"),
    ("James", "Monroe", "1817-03-04", "Democratic-Republican"),
    ("James", "Monroe", "1821-03-04", "Democratic-Republican"),
    ("John Quincy", "Adams", "1825-03-04", "Democratic-Republican"),
]

df = spark.createDataFrame(data, ["first_name", "last_name", "start_date", "party"])
df.show()

+-----------+----------+----------+--------------------+
| first_name| last_name|start_date|               party|
+-----------+----------+----------+--------------------+
|     George|Washington|1789-04-30|                null|
|       John|     Adams|1797-03-04|          Federalist|
|      James|   Madison|1809-03-04|Democratic-Republ...|
|      James|   Madison|1813-03-04|Democratic-Republ...|
|      James|    Monroe|1817-03-04|Democratic-Republ...|
|      James|    Monroe|1821-03-04|Democratic-Republ...|
|John Quincy|     Adams|1825-03-04|Democratic-Republ...|
+-----------+----------+----------+--------------------+



This dataframe lists US Presidents along with their party and start date. Although, presidents that served multiple terms appear in multiple rows.

The **firstInGroup()** function will remove the duplicate rows, based on the `group_by` columns, retaining the first row of the group when ordered by the `sort_by` column. In this example, we group by both `first_name` and `last_name`, then sort by `start_date` to retain only the first term for each President.

In [14]:
df.firstInGroup(["first_name", "last_name"], F.col("start_date")).show()

+-----------+----------+----------+--------------------+
| first_name| last_name|start_date|               party|
+-----------+----------+----------+--------------------+
|     George|Washington|1789-04-30|                null|
|      James|   Madison|1809-03-04|Democratic-Republ...|
|      James|    Monroe|1817-03-04|Democratic-Republ...|
|       John|     Adams|1797-03-04|          Federalist|
|John Quincy|     Adams|1825-03-04|Democratic-Republ...|
+-----------+----------+----------+--------------------+

